## Imports & Inputs

In [1]:
from pathlib import Path
from typing import Optional

import git

REPO_ROOT = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PUZZLE_INPUT = REPO_ROOT / 'puzzle_inputs' / 'Day 10.txt'

assert PUZZLE_INPUT.exists()

<h2>--- Day 10: Syntax Scoring ---</h2><p>You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:</p>
<pre><code>Syntax error in navigation subsystem on line: <span title="Some days, that's just how it is.">all of them</span></code></pre>
<p><em>All of them?!</em> The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).</p>
<p>The navigation subsystem syntax is made of several lines containing <em>chunks</em>. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must <em>open</em> and <em>close</em> with one of four legal pairs of matching characters:</p>
<ul>
<li>If a chunk opens with <code>(</code>, it must close with <code>)</code>.</li>
<li>If a chunk opens with <code>[</code>, it must close with <code>]</code>.</li>
<li>If a chunk opens with <code>{</code>, it must close with <code>}</code>.</li>
<li>If a chunk opens with <code>&lt;</code>, it must close with <code>&gt;</code>.</li>
</ul>
<p>So, <code>()</code> is a legal chunk that contains no other chunks, as is <code>[]</code>. More complex but valid chunks include <code>([])</code>, <code>{()()()}</code>, <code>&lt;([{}])&gt;</code>, <code>[&lt;&gt;({}){}[([])&lt;&gt;]]</code>, and even <code>(((((((((())))))))))</code>.</p>
<p>Some lines are <em>incomplete</em>, but others are <em>corrupted</em>. Find and discard the corrupted lines first.</p>
<p>A corrupted line is one where a chunk <em>closes with the wrong character</em> - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.</p>
<p>Examples of corrupted chunks include <code>(]</code>, <code>{()()()&gt;</code>, <code>(((()))}</code>, and <code>&lt;([]){()}[{}])</code>. Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.</p>
<p>For example, consider the following navigation subsystem:</p>
<pre><code>[({(&lt;(())[]&gt;[[{[]{&lt;()&lt;&gt;&gt;
[(()[&lt;&gt;])]({[&lt;{&lt;&lt;[]&gt;&gt;(
{([(&lt;{}[&lt;&gt;[]}&gt;{[]{[(&lt;()&gt;
(((({&lt;&gt;}&lt;{&lt;{&lt;&gt;}{[]{[]{}
[[&lt;[([]))&lt;([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{&lt;[[]]&gt;}&lt;{[{[{[]{()[[[]
[&lt;(&lt;(&lt;(&lt;{}))&gt;&lt;([]([]()
&lt;{([([[(&lt;&gt;()){}]&gt;(&lt;&lt;{{
&lt;{([{{}}[&lt;[[[&lt;&gt;{}]]]&gt;[]]
</code></pre>
<p>Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:</p>
<ul>
<li><code>{([(&lt;{}[&lt;&gt;[]}&gt;{[]{[(&lt;()&gt;</code> - Expected <code>]</code>, but found <code>}</code> instead.</li>
<li><code>[[&lt;[([]))&lt;([[{}[[()]]]</code> - Expected <code>]</code>, but found <code>)</code> instead.</li>
<li><code>[{[{({}]{}}([{[{{{}}([]</code> - Expected <code>)</code>, but found <code>]</code> instead.</li>
<li><code>[&lt;(&lt;(&lt;(&lt;{}))&gt;&lt;([]([]()</code> - Expected <code>&gt;</code>, but found <code>)</code> instead.</li>
<li><code>&lt;{([([[(&lt;&gt;()){}]&gt;(&lt;&lt;{{</code> - Expected <code>]</code>, but found <code>&gt;</code> instead.</li>
</ul>
<p>Stop at the first incorrect closing character on each corrupted line.</p>
<p>Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the <em>first illegal character</em> on the line and look it up in the following table:</p>
<ul>
<li><code>)</code>: <code>3</code> points.</li>
<li><code>]</code>: <code>57</code> points.</li>
<li><code>}</code>: <code>1197</code> points.</li>
<li><code>&gt;</code>: <code>25137</code> points.</li>
</ul>
<p>In the above example, an illegal <code>)</code> was found twice (<code>2*3 = <em>6</em></code> points), an illegal <code>]</code> was found once (<code><em>57</em></code> points), an illegal <code>}</code> was found once (<code><em>1197</em></code> points), and an illegal <code>&gt;</code> was found once (<code><em>25137</em></code> points). So, the total syntax error score for this file is <code>6+57+1197+25137 = <em>26397</em></code> points!</p>
<p>Find the first illegal character in each corrupted line of the navigation subsystem. <em>What is the total syntax error score for those errors?</em></p>


## Solution

In [2]:
def first_illegal_character(line: str) -> Optional[str]:
    matching_opening_delimiter = dict(zip(')]}>', '([{<'))
    opening_delimiters = set(matching_opening_delimiter.values())

    delimiter_stack = []
    for delimiter in line:
        if delimiter in opening_delimiters:
            delimiter_stack.append(delimiter)
        elif delimiter_stack.pop() != matching_opening_delimiter[delimiter]:
            return delimiter


def score_syntax_errors(navigation_subsystem_source: str) -> int:
    score_for_first_illegal_delimiter_per_line = {
        None: 0,
        ')': 3,
        ']': 57,
        '}': 1197,
        '>': 25137
    }
    return sum(
        score_for_first_illegal_delimiter_per_line[first_illegal_character(line)]
        for line in navigation_subsystem_source.split('\n')
    )

### Testing

In [3]:
assert first_illegal_character('()') is None
assert first_illegal_character('[]') is None
assert first_illegal_character('([])') is None
assert first_illegal_character('{()()()}') is None
assert first_illegal_character('<([{}])>') is None
assert first_illegal_character('[<>({}){}[([])<>]]') is None
assert first_illegal_character('(((((((((())))))))))') is None

assert first_illegal_character('{([(<{}[<>[]}>{[]{[(<()>') == '}'
assert first_illegal_character('[[<[([]))<([[{}[[()]]]') == ')'
assert first_illegal_character('[{[{({}]{}}([{[{{{}}([]') == ']'
assert first_illegal_character('[<(<(<(<{}))><([]([]()') == ')'
assert first_illegal_character('<{([([[(<>()){}]>(<<{{') == '>'

example_navigation_subsystem_source = '\n'.join([
    '({(<(())[]>[[{[]{<()<>>',
    '[(()[<>])]({[<{<<[]>>(',
    '{([(<{}[<>[]}>{[]{[(<()>',
    '(((({<>}<{<{<>}{[]{[]{}',
    '[[<[([]))<([[{}[[()]]]',
    '[{[{({}]{}}([{[{{{}}([]',
    '{<[[]]>}<{[{[{[]{()[[[]',
    '[<(<(<(<{}))><([]([]()',
    '<{([([[(<>()){}]>(<<{{',
    '<{([{{}}[<[[[<>{}]]]>[]]'
])

assert score_syntax_errors(example_navigation_subsystem_source) == 26397

### Answer

In [4]:
print(
    'Syntax errors score:',
    score_syntax_errors(PUZZLE_INPUT.read_text())
)

Syntax errors score: 362271
